In [1]:
from haystack import Pipeline, component
from haystack_experimental.chat_message_stores.in_memory import InMemoryChatMessageStore
from haystack_experimental.components.retrievers import ChatMessageRetriever
from haystack_experimental.components.writers import ChatMessageWriter
from haystack.components.embedders import SentenceTransformersTextEmbedder
from haystack.components.builders import ChatPromptBuilder
from haystack.components.generators.chat import OpenAIChatGenerator
from haystack_integrations.document_stores.mongodb_atlas import MongoDBAtlasDocumentStore
from haystack_integrations.components.retrievers.mongodb_atlas import MongoDBAtlasEmbeddingRetriever
from haystack.utils import Secret
from haystack.dataclasses import ChatMessage
import os
from getpass import getpass

/Users/mhdfarhanali/Documents/Farhan SmartShopper/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
os.environ["MONGO_CONNECTION_STRING"] = getpass("Masukkan MongoDB Connection String Anda: ")
os.environ["OPENAI_API_KEY"] = getpass("Masukkan OpenAI API Key Anda: ")

In [3]:
# Initialize MongoDB Document Store

document_store = MongoDBAtlasDocumentStore(
    database_name="depato_store",
    collection_name="common_information",
    vector_search_index="vector_index_common",
    full_text_search_index=None
)

print("Connected to MongoDB Atlas successfully!")

Connected to MongoDB Atlas successfully!


In [4]:
# Chat Message Store (for memory)
chat_memory = InMemoryChatMessageStore()
memory_retriever = ChatMessageRetriever(chat_memory)
memory_writer = ChatMessageWriter(chat_memory)

In [5]:
# Define Prompt Template

system_message = ChatMessage.from_system(
    "You are a friendly and professional customer support assistant. Use context and chat history to help the user."
)

user_template = """
Context:
{% for doc in documents %}
{{ doc.content }}
{% endfor %}

Previous chat:
{% for mem in memories %}
{{ mem.sender }}: {{ mem.text }}
{% endfor %}

Question: {{query}}
Answer clearly and politely:
"""
user_message = ChatMessage.from_user(user_template)

In [6]:
# Build Pipeline
pipeline = Pipeline()

pipeline.add_component("memory_retriever", memory_retriever)
pipeline.add_component("embedder", SentenceTransformersTextEmbedder(model="sentence-transformers/all-mpnet-base-v2"))
pipeline.add_component("retriever", MongoDBAtlasEmbeddingRetriever(document_store=document_store, top_k=5))
pipeline.add_component(
    "prompt_builder",
    ChatPromptBuilder(variables=["query", "documents", "memories"], required_variables=["query", "documents", "memories"])
)
pipeline.add_component("generator", OpenAIChatGenerator(model="gpt-4.1", api_key=Secret.from_token(os.environ["OPENAI_API_KEY"])))
pipeline.add_component("memory_writer", memory_writer)

# Connect components
pipeline.connect("memory_retriever", "prompt_builder.memories")
pipeline.connect("embedder", "retriever")
pipeline.connect("retriever", "prompt_builder.documents")
pipeline.connect("prompt_builder.prompt", "generator.messages")
pipeline.connect("generator.replies", "memory_writer.messages")

print("Common Chat Memory Assistant pipeline built successfully!")

Common Chat Memory Assistant pipeline built successfully!


In [7]:
# Chat Loop

while True:
    query = input("\nYou: ")
    if query.lower() in ["exit", "quit", "bye"]:
        print("Goodbye!")
        break

    messages = [system_message, user_message]

    response = pipeline.run(
        {
            "embedder": {"text": query},
            "prompt_builder": {"query": query, "template": messages}
        },
        include_outputs_from=["generator"]
    )

    answer = response["generator"]["replies"][0].text
    print(f"AI: {answer}")

Batches: 100%|██████████| 1/1 [00:00<00:00,  3.83it/s]


AI: We accept a variety of payment methods, including credit and debit cards, bank transfers, e-wallets, and Cash on Delivery (COD) for eligible areas. If you need more details or have a specific payment method in mind, please let us know!


Batches: 100%|██████████| 1/1 [00:00<00:00, 11.68it/s]


AI: You can request a refund within 7 days of receiving your order. Simply go to your order history page or contact our customer support team, and we’ll assist you with the process. If you have any questions or need help with your refund, please let us know!
Goodbye!
